# Seasonalities and Holidays
Despite that time-series modeling is not super feature-rich, we can still leverage at least deterministic features which are known in advance - calendar patterns. 

The easiest way how to model seasonality and leverage the power of ML models is to create a binary feature for each repeating calendar pattern (day of the week, the month of the year...). This is implemented in the 
`SeasonalityTransformer`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from hcrystalball.utils import get_sales_data

df = get_sales_data(n_dates=365*2, 
                    n_assortments=1, 
                    n_states=1, 
                    n_stores=1)[:"2014-12-31"]
X, y = pd.DataFrame(index=df.index), df['Sales']

## Seasonality Features

In [ ]:
from hcrystalball.feature_extraction import SeasonalityTransformer
SeasonalityTransformer(freq = 'D').fit(X, y).transform(X).head()

## Holiday Features

One of the important parts of time-series modeling is taking into account holidays which in most cases exhibit different behavior than it's common. hcrystalball implements HolidayTransformer which returns column with string name of the holiday based on provided country ISO code for each date in the dataset (empty string if it's not a holiday). All hcrystalball wrappers accept the output of `HolidayTransformer` and transform it into individual model formats. `HolidayTransformer` also supports some region specific holidays i.e. Germany state specific holidays, in that case the provided string shoule be in form country-region: i.e. 'DE-HE'.

In [ ]:
from hcrystalball.feature_extraction import HolidayTransformer
HolidayTransformer(country_code = 'DE').fit(X, y).transform(X).tail(10)

The holiday effects have very often impact on target variable even before or after holidays (i.e. whole Christmas period is affected by few public holidays), in order to model such effects around holiday `HolidayTransformer` provides 3 parameters - `days_before` - number of days the before public holiday which should be taken into account, `days_after` - number of days before the public holiday which should be taken into account and bool variable `bridge_days` which will create variable for the overlaping `days_before` and `days_after` feature - mostly for modeling working days in the middle of two public holidays.

In [ ]:
from hcrystalball.feature_extraction import HolidayTransformer
HolidayTransformer(country_code = 'DE', days_before = 2, days_after = 1, bridge_days = True).fit(X, y).transform(X)['2014-04-16':'2014-04-23']

## In Pipelines

It is possible to combine more countries/regions for the holidays. The behovior of multi holidays is wrapper specific.

In [ ]:
from hcrystalball.compose import TSColumnTransformer
from hcrystalball.wrappers import ExponentialSmoothingWrapper
from hcrystalball.wrappers import get_sklearn_wrapper
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from hcrystalball.wrappers import SarimaxWrapper

In [ ]:
pipeline = Pipeline([
    ('holidays_de', HolidayTransformer(country_code = 'DE')),
    ('holidays_be', HolidayTransformer(country_code = 'BE')),
    ('seasonality', SeasonalityTransformer(freq='D')),
    ('model', get_sklearn_wrapper(RandomForestRegressor, random_state=42))
]) 

In [ ]:
preds = (pipeline.fit(X[:-10], y[:-10])
         .predict(X[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50))

preds.plot(title=f"MAE:{(preds['Sales']-preds['sklearn']).abs().mean().round(3)}");

### Also country code column can be used. 
*Note: This column is deleted in `HolidayTransformer.transform` method in order not to pollute the further processing*

In [ ]:
pipeline_col = Pipeline([
    ('holidays_de', HolidayTransformer(country_code_column = 'germany')),
    ('holidays_be', HolidayTransformer(country_code_column = 'belgium')),    
    ('seasonality', SeasonalityTransformer(freq='D')),
    ('model', get_sklearn_wrapper(RandomForestRegressor, random_state=42))
]) 

In [ ]:
X_col = X.copy().assign(germany='DE').assign(belgium='BE')

In [ ]:
preds_col = (pipeline_col.fit(X_col[:-10], y[:-10])
         .predict(X_col[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50))

preds_col.plot(title=f"MAE:{(preds_col['Sales'] - preds_col['sklearn']).abs().mean().round(3)}");

## With exogenous variables

In [ ]:
X_col['trend'] = np.arange(len(X))

In [ ]:
preds_col = (pipeline_col.fit(X_col[:-10], y[:-10])
         .predict(X_col[-10:])
         .merge(y, left_index=True, right_index=True, how='outer')
         .tail(50))

preds_col.plot(title=f"MAE:{(preds_col['Sales'] - preds_col['sklearn']).abs().mean().round(3)}");

### Individual holidays as separate columns

In [ ]:
sklearns_native_transformers = TSColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(sparse=False, drop='first'), ['_holiday_CZ'])
    ])    

pipeline_col = Pipeline([
   ('holidays', HolidayTransformer(country_code='CZ')),    
    ('one_hot_encoder', sklearns_native_transformers)
]) 

In [ ]:
pipeline_col.fit_transform(X_col['2013-12-20':'2013-12-26'])